In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from nba_api.stats.static import players
from nba_api.stats.static import teams

In [4]:
all_players = pd.DataFrame(players.get_players())
all_players = all_players[all_players['is_active'] == True]
all_active_players = all_players.drop(['is_active'],axis=1)
all_active_players.head(2)

,id,full_name,first_name,last_name
21,203500,Steven Adams,Steven,Adams
23,1628389,Bam Adebayo,Bam,Adebayo


In [5]:
all_teams = pd.DataFrame(teams.get_teams())
all_teams.head(2)

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946


In [6]:
from nba_api.stats.library.parameters import SeasonAll
from nba_api.stats.endpoints import playergamelog

In [9]:
gamelog_bron = playergamelog.PlayerGameLog(player_id='2544',season='2021-22')
bron_df = gamelog_bron.get_data_frames()[0]

In [10]:
bron_df['PTS'].mean()

30.267857142857142

In [11]:
from nba_api.stats.endpoints import leaguegamefinder
GSW_games = leaguegamefinder.LeagueGameFinder(team_id_nullable='1610612744',season_nullable='2020-21').get_data_frames()[0]

In [24]:
all_teams.head(2)

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946


In [30]:
def team_games(team,season):
    team_id = all_teams[all_teams['abbreviation'] == team]['id']
    team_games = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id,season_nullable=season).get_data_frames()[0]
    return team_games

team_games('NYK','2020-21').head(2)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42020,1610612752,NYK,New York Knicks,0042000135,2021-06-02,NYK vs. ATL,L,240,89,...,0.833,14,31,45,19,5,4,16,22,-14.0
1,42020,1610612752,NYK,New York Knicks,0042000134,2021-05-30,NYK @ ATL,L,242,96,...,0.826,6,33,39,24,7,2,9,20,-17.0


In [13]:
GSW_games.head(2)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,52020,1610612744,GSW,Golden State Warriors,0052000211,2021-05-21,GSW vs. MEM,L,265,112,...,0.800,11,37,48,30,7,7,21,23,-5.0
1,52020,1610612744,GSW,Golden State Warriors,0052000121,2021-05-19,GSW @ LAL,L,240,100,...,0.733,9,37,46,19,8,8,20,21,-3.0


In [14]:
GSW_games['Opts'] = GSW_games['PTS'] - GSW_games['PLUS_MINUS']

In [32]:
GSW_games.head(2)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,Opts
0,52020,1610612744,GSW,Golden State Warriors,0052000211,2021-05-21,GSW vs. MEM,L,265,112,...,11,37,48,30,7,7,21,23,-5.0,117.0
1,52020,1610612744,GSW,Golden State Warriors,0052000121,2021-05-19,GSW @ LAL,L,240,100,...,9,37,46,19,8,8,20,21,-3.0,103.0


In [47]:
def create_df(game_id):
    # Get **all** the games so we can filter to an individual GAME_ID
    result = leaguegamefinder.LeagueGameFinder()
    all_games = result.get_data_frames()[0]
    # Find the game_id we want
    game_id = '0052000121'
    full_game1 = all_games[all_games.GAME_ID == game_id]
    return full_game1

create_df('0052000121')

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
6285,52020,1610612747,LAL,Los Angeles Lakers,0052000121,2021-05-19,LAL vs. GSW,W,240,103,...,0.760,13,36,49,27,15,7,11,18,3.0
6290,52020,1610612744,GSW,Golden State Warriors,0052000121,2021-05-19,GSW @ LAL,L,240,100,...,0.733,9,37,46,19,8,8,20,21,-3.0


In [41]:
def combine_team_games(df, keep_method='home'):
    
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result
    
# Combine the game rows into one. By default, the home team will be TEAM_A.
game_df = combine_team_games(full_game)
game_df

,SEASON_ID,TEAM_ID_A,TEAM_ABBREVIATION_A,TEAM_NAME_A,GAME_ID,GAME_DATE,MATCHUP_A,WL_A,MIN_A,PTS_A,...,FT_PCT_B,OREB_B,DREB_B,REB_B,AST_B,STL_B,BLK_B,TOV_B,PF_B,PLUS_MINUS_B
1,52020,1610612747,LAL,Los Angeles Lakers,0052000121,2021-05-19,LAL vs. GSW,W,240,103,...,0.733,9,37,46,19,8,8,20,21,-3.0


In [ ]:
gsw_vs_cle_id
gsw_vs_cle = combine_team_games()

In [18]:
def get_team_games(team, season1):
    team_id = all_teams[all_teams['abbreviation'] == team]['id']
    df = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id, season_nullable=season1).get_data_frames()[0]
    return df

In [23]:
team_select = 'CLE'
season_select = '2016-17'

team_df = get_team_games(team_select,season_select)
team_df.head(2)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42016,1610612739,CLE,Cleveland Cavaliers,0041600405,2017-06-12,CLE @ GSW,L,240,120,...,0.652,12,28,40,22,6,5,14,22,-9.0
1,42016,1610612739,CLE,Cleveland Cavaliers,0041600404,2017-06-09,CLE vs. GSW,W,240,137,...,0.677,11,30,41,27,6,3,11,24,19.0


# create streamlit page with this data

Create a selector for team -> player -> stats
show dat per season for selected player - same as it is in nba stats page for the player

i want to create some viz for the player:
    - shot chart
    - +/- per season
    - (check online what other viz)